# Research Project

------------------------------------------------------

## Neural Networks

**Enrique Botía Barberá** - 100406632






# Introduction
For this project was required to select a paper about a deep learning topic and reproduce one or more illustrative experiments. I have chosen the paper [Deep Learning for Symbolic Mathematics](https://arxiv.org/pdf/1912.01412v1.pdf), using the code in the github repository [SymbolicMathematics](https://github.com/facebookresearch/SymbolicMathematics), developed by facebook. In the following I will explain the content of the paper and then show its implementation and some experiments that I have performed.

# Explanation of the paper
The authors face the challenge to map symbolic reasoning into the real space. Two symbolic mathematical problems are considered: symbolic integrations and differential equations.The content of the paper is focused mainly in how to represent and generate the data to train the model, to later apply a seq2seq model to predict the solution of the given operation.

## Mathematics as a natural language
Trees are a suitable representation of mathematical expressions, but tree-to-tree models are more involved and slower than seq2seq models. So, to get the dataset (composed by the problems and its corresponding solutions) random trees are generated and then converted to the prefix notation, e.g. the expression "$2 + 3∗(5+ 2)$" in the prefix notation is represented as the sequence "$[+ 2 ∗ 3 + 5\,2]$" and also as a tree:

<img src='https://drive.google.com/uc?id=1uILwHqYen9QoGTuaFFHkBgtXB-EcwOce'>

The algorithm that they implement to compute random trees with equal probability can be found in the appendix of the paper.


They also analyse the number all of possible to trees, under different constraints. This will allow them to know the needed size the sample, so that it is representative in terms of the total problem space. 

## Generating datasets
For the task handled, solutions of random problems sometimes cannot be easily derived or don't exist. Because of this, it is need to design a specialized way to generate the data in each of the problems.

### Integration
- **Foward generation (FWD)**: generate random expressions with the algorithm that is showed in the paper and solve the integrations with an external math framework.
- **Backward generation (BWD)**: as FWD, but computing the derivative instead, which is always possible, faster and independent of external resources. Once computed, the derivative is added the dataset as the problem and the original expression as its corresponding solution.
- **Backward generation with integration by parts (IBP)**: use integrations by parts to generate examples only by derivating, as FWD.

In FWD and IBP the integral (the solution) tends to be longer than the derivative (the problem), while BWD generation favors the opposite. So, a mixture of BWD and IBP generated data should therefore provide a better representation of problem space, without resorting to external tools.

### ODEs
The procedure followed can be understood with the examples that are presented in the paper.
For the first order differential equation (ODE 1):
<img src='https://drive.google.com/uc?id=1NzQHeMba52XxgiUB5l9YqPUYdG1MUQ_d'>

And for the second order differential equation (ODE 2):

<img src='https://drive.google.com/uc?id=1Ysn4IAFJ0M5a2o3NkE4fSDNXTAXlGFGh'>


## Experiments performed by the authors of the paper
### Dataset
The datasets created were restricted to a maximum size, only one variable, and integer numbers in the region {-5,...,5}.
### Model
It was used a [transformer model](https://arxiv.org/pdf/1706.03762.pdf) with 8 attention heads, 6 layers, and a dimensionality of 512. They used the Adam optimizer with a learning rate of $10^{-4}$, and 256 equations per batch. At inference, expressions are generated by a beam search, with early stopping. They normalize the log-likelihood scores of the hypotheses in the beam by their sequence length. Results were reported with beam widths of 1, 10 and 50.

## Evaluation
Since the correctness of generated expression can be easily verifyed (only it is needed to derivate if the problem was an integrations or susbtitute if it was a differential equation) all hypotheses in the beam are checked, if one of them is correct means that the model has successfully solved the probem.

## Results
This table show the accuracies obtained, also by comparing with other popular external frameworks:

![results](https://drive.google.com/uc?id=1fQ8gyobxLrfqUsNAc-AMquvWoNdR0jLe)

# Implementation
After having explained the contents of the paper, I attach here the the code of the official github repository, adding some explanations in each part. Then I have implemented a couple of experiments taking advantage of this code. This code is required to be executed using the gpu.
## Setting the enviroment

In [1]:
import os
if 'google.colab' in str(get_ipython()):
    # mount drive (it's required be located in the folder where you have all the needed files)
    from google.colab import drive
    drive.mount("/content/drive")
    
# input folder where you have store the needed files
local_folder = input("Enter here your local folder: ")
%cd $local_folder
%ls
assert os.path.isfile('fwd_bwd.pth')
assert os.path.isfile('ode1.pth')

Mounted at /content/drive
Enter here your local folder: drive/MyDrive/UNI/3º/Colab Research Project/
/content/drive/MyDrive/UNI/3º/Colab Research Project
 fwd_bwd.pth   img/   ode1.pth  'Research Project.ipynb'   src/


In [2]:
# importing the needed libraries
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

## Code from SymbolicMathematics repository by facebookresearch

### Build environment / Reload model

In [3]:
# setting the parameters
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': './fwd_bwd.pth',

})

In [4]:
env = build_env(params)
x = env.local_dict['x']

In [5]:
#torch.load(map_location=torch.device('cpu'))
modules = build_modules(env, params)

encoder = modules['encoder']
decoder = modules['decoder']

### Start from a function F, compute its derivative f = F', which  will be tried to recover F from f

In [6]:
# here you can modify the integral function the model has to predict, F
F_infix = 'x * tan(exp(x)/x)'
F_infix = 'x * cos(x**2) * tan(x)'
F_infix = 'cos(x**2 * exp(x * cos(x)))'
F_infix = 'ln(cos(x + exp(x)) * sin(x**2 + 2) * exp(x) / x)'

In [7]:
# F (integral, that the model will try to predict)
F = sp.S(F_infix, locals=env.local_dict)
F

log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)

In [8]:
# f (F', that the model will take as input)
f = F.diff(x)
f

x*(2*exp(x)*cos(x + exp(x))*cos(x**2 + 2) - (exp(x) + 1)*exp(x)*sin(x + exp(x))*sin(x**2 + 2)/x + exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x - exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x**2)*exp(-x)/(sin(x**2 + 2)*cos(x + exp(x)))

### Compute prefix representations

In [9]:
F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
print(f"F prefix: {F_prefix}")
print(f"f prefix: {f_prefix}")

F prefix: ['ln', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2']
f prefix: ['mul', 'x', 'mul', 'pow', 'cos', 'add', 'x', 'exp', 'x', 'INT-', '1', 'mul', 'pow', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'INT-', '1', 'mul', 'add', 'mul', 'INT+', '2', 'mul', 'cos', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'exp', 'x', 'add', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'add', 'INT+', '1', 'exp', 'x', 'mul', 'exp', 'x', 'mul', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'sin', 'add', 'x', 'exp', 'x', 'exp', 'mul', 'INT-', '

### Encode input

In [10]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
# note that the expressions are converted to ids to improve the process
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

### Decode with beam search

In [11]:
beam_size = 10
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### Print results

In [12]:
print("Input function f:\n")
display(f)
print("\nReference function F:\n")
display(F)
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        # note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it may be correct
        res = "OK" if simplify(hyp.diff(x) - f, seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
    
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function f:



x*(2*exp(x)*cos(x + exp(x))*cos(x**2 + 2) - (exp(x) + 1)*exp(x)*sin(x + exp(x))*sin(x**2 + 2)/x + exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x - exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x**2)*exp(-x)/(sin(x**2 + 2)*cos(x + exp(x)))


Reference function F:



log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)


-0.00003  OK  log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)
-0.28475  OK  log(exp(x)*sin((x**3 + 2*x)/x)*cos(x + exp(x))/x)
-0.28592  OK  log(exp(x)*sin(x*(x + 2/x))*cos(x + exp(x))/x)
-0.35794  OK  log(exp(x)*sin(x*(x + 1) - x + 2)*cos(x + exp(x))/x)
-0.37952  NO  log(exp(x)*sin(x**2*(x + 2/x))*cos(x + exp(x))/x)
-0.38034  NO  log(exp(x)*sin(x**2 + 2)*cos(x + sinh(x) + cosh(x))/x)
-0.39518  OK  atan(tan(log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)))
-0.39689  OK  log(exp(x)*sin(x*(x - 1) + x + 2)*cos(x + exp(x))/x)
-0.43203  NO  log(exp(x)*sin((x**2 + 2)**2)*cos(x + exp(x))/x)
-0.44538  NO  log(exp(x)*sin(x**2 + 2*x)*cos(x + exp(x))/x)

The well-predicted expression with the highest score is:



log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)

## Own experiments

### Exploiding the equivalent solution property

It is also mentioned in the paper that this model has the interesting property of generating solutions that are exactly equivalent, but written in different way (it can be observed in the results of the previous code examples). So, without having been trained to do so, the model is able to recover equivalent expressions. Considering this, I have implemented a fragment of code in which it is requested an input expresion and returns you the equivalent expressions in the SimPy format.

Here I attach the code:

In [13]:
# F (integral, that the model will try to predict)
F = input("You want to find equivalent expression from: ")
# I executed this cell for F=x**2+2*x+a8 (where a8 is a constant)
F = sp.S(F, locals=env.local_dict)
F_prefix = env.sympy_to_prefix(F)
print("Your input expression (that is the problem solution) is:")
display(F)
# f (F', that the model will take as input)
f = F.diff(x)
print("Your derivative of your problem (that is the problem) is:")
display(f)
print("")

F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

beam_size = 50
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

print("Printing equivalent expressions...")
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):
    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix
    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy
        # check whether we recover f if we differentiate the hypothesis
        if simplify(hyp.diff(x) - f, seconds=1)==0:
          print("")
          display(hyp)
          if (input("\n\nDo you want to display another equivalent expression (Y/y)? ").lower()!="y"):
            break

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

You want to find equivalent expression from: x**2+2*x+a8
Your input expression (that is the problem solution) is:


a8 + x**2 + 2*x

Your derivative of your problem (that is the problem) is:


2*x + 2


Printing equivalent expressions...



...



Do you want to display another equivalent expression (Y/y)? Y



-x**2 + x*(2*x + 2)



Do you want to display another equivalent expression (Y/y)? y



(x**3 + 2*x**2 + x)/x



Do you want to display another equivalent expression (Y/y)? n



### Playing with ODEs
The paper shows, in the results of the experiments performed by the authors, that the accuracy obtained by the model is greater than the one obtained with the mathematica software. I had curiosity about this fact and ask to a friend, who has finished the degree of physics, for differential equations of first order that are not to solvable by mathematica (they are declared below). Will be this model capable to find their solutions?

In [14]:
# changing some parameters
params["tasks"] = 'ode1'
params["reload_model"] = './ode1.pth'
params["n_variables"] = 2

# building the enviroment
env = build_env(params)
x = env.local_dict['x']
y = env.local_dict['y']
f = env.local_dict['f']
a8 = env.local_dict['a8']

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

In [15]:
# this is an easy ODE problem just for testing
ode = x*sp.Derivative(f(x))-f(x)+x
# these are all the equations not solvable by mathematica (you can choose anyone):
ode = (-x*sp.cos(x)*f(x)+sp.sec(x))/(sp.cos(x)-2*x*sp.sin(x))-sp.Derivative(f(x))
ode = (1-x*sp.cos(x)*f(x))/(sp.cos(x)-x*sp.sin(x))-sp.Derivative(f(x))
ode = (sp.exp(-x)-x*f(x))/(1+x)-sp.Derivative(f(x))
ode = (sp.sqrt((1+x**2))-(1+x**2)*f(x))/x-sp.Derivative(f(x))
ode = sp.Derivative(f(x))+x*f(x)+a8
ode = 1-x*f(x)-sp.Derivative(f(x))

In [16]:
ode_prefix = env.sympy_to_prefix(ode)
x1_prefix = env.clean_prefix(ode_prefix)

x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

beam_size = 10
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

In [17]:
print("Input function ODE:\n")
display(ode)
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        res = "OK" if simplify(simplify(ode.subs(f(x), hyp), seconds=1), seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
        
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function ODE:



-x*f(x) - Derivative(f(x), x) + 1


-0.04524  NO  a8*exp(-x**2/2) + 1
-0.08929  NO  exp(a8 - x**2/2) + 1
-0.13985  NO  (a8*x*exp(-x**2/2) + x)/x
-0.15581  NO  1 - exp(a8 - x**2/2)
-0.19425  NO  a8*exp(-x*(x + 1)/2 + x/2) + 1
-0.19915  NO  exp(x*(a8/x - x)/2) + 1
-0.20747  NO  (x*exp(a8 - x**2/2) + x)/x
-0.23801  NO  a8*exp(-(x**3 + x)/(2*x)) + 1
-0.27655  NO  a8*exp(-(x**3 + 2*x)/(2*x)) + 1
-0.28619  NO  a8*exp(-x**2/2 + pi/4) + 1

The well-predicted expression with the highest score is:



None

We can observe that the model is not able to solve the problem.

Unfortunately, the result is the same for the rest of the expressions that my friend told me. My final conclusion is that the model is accurate (to the point to be can be comparable with mathematica), but only in a range of problems that, although really wide, is limited considering all the possible ones that could be considered in real life, in contrast to other mathematical frameworks that are of general pourpuse.